<a href="https://colab.research.google.com/github/asigalov61/DOREMI/blob/main/REMI_MIDI_Encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install miditoolkit
import miditoolkit
!git clone https://github.com/asigalov61/remi.git
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.chdir('remi')

import utils

In [ ]:
#@title (Best Choice/Works best stand-alone) Alex Piano Only Drafts Original 1500 MIDIs 
!wget 'https://github.com/asigalov61/AlexMIDIDataSet/raw/master/AlexMIDIDataSet-CC-BY-NC-SA-All-Drafts-Piano-Only.zip'
!unzip -j 'AlexMIDIDataSet-CC-BY-NC-SA-All-Drafts-Piano-Only.zip'

In [ ]:
def get_events_from_midis(midipath):
  note_items, tempo_items = utils.read_items(str(midipath))
  note_items = utils.quantize_items(note_items)
  chord_items = utils.extract_chords(note_items)
  items = chord_items + tempo_items + note_items
  max_time = note_items[-1].end
  groups = utils.group_items(items, max_time)
  midievents = utils.item2event(groups)
  return midievents

In [ ]:
#@title Calculate Number of Indexable MIDI files :)
import os
from os import listdir
from os.path import isfile, join

# Modify these paths to your own paths to MAESTRO dataset
maestro_dir = '/content/remi'

data_dirs = ['/content/remi']

data_files = []
for data_dir in data_dirs:
    data_files += [join(data_dir, f) for f in listdir(data_dir) if isfile(join(data_dir, f)) if 'mid' in f] # MIDI files can have MIDI and MID extensions. Remove/comment the one you do not need/use

data_files.sort()

print('total midi/mid files : ', len(data_files))

In [ ]:
from tqdm import tqdm_notebook as tqdm
import os
import numpy as np

dataset_dir = 'dataset'

if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)

for i in tqdm(range(len(data_files))):
    print(data_files[i])
    eventlist = get_events_from_midis(data_files[i])
    #eventlist = events
    print(eventlist)
    
    save_file = dataset_dir + '/' + str(i)
    data = {'eventlist': eventlist}
    np.savez(save_file, **data, allow_pickle=False)
    #np.savetxt(save_file, eventlist, delimiter=',')